In [1]:
import os
import numpy as np
import pandas as pd
import glob

Step 1. 형태소 분석기 변경해 보기 - Mecab 이외에도 Hannanum, Kkma, Komoran, Okt


Step 2. 불용어 추가해 보기 - 직접 뉴스를 열어보면서 불용어가 총 50개가 될 때까지 추가, 추가 전후 모델성능 비교


Step 3. 다른 날짜 데이터 추가해 보기 - 미 수집한 2020년 5월 6일 데이터와 추가로 수집된 데이터를 적절히 합치려면? 적절히 데이터를 섞기위해 수집고민하기

# 데이터 가져오기

In [2]:
glob.glob(os.getenv('HOME')+'/aiffel/news_crawler/*.csv')

['/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200215.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200530.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200914.csv',
 '/home/jin/aiffel/news_crawler/news_data.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200429.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200129.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200819.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200614.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20201024.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200715.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20200315.csv',
 '/home/jin/aiffel/news_crawler/news_20_[101, 102, 103, 105]_20201119.csv']

In [3]:
# map(func, iterable) 후 pd.concat로 DataFrame결합
df = pd.concat(map(pd.read_csv, glob.glob(os.getenv('HOME')+'/aiffel/news_crawler/*.csv')))

In [4]:
os.getcwd()

'/home/jin/workplace'

In [5]:
df

,news,code
0,[파이낸셜뉴스] 고가주택에 대한 대출규제를 담은 12.16대책 이후 서울 아파트 시...,경제
1,플룸테크 JTI코리아 제공 플룸테크 JTI코리아 제공\n\n[파이낸셜뉴스] 전자담배...,경제
2,[파이낸셜뉴스] 이아이디가 2차전지 자회사의 실적호전에 힘입어 지난해 큰 폭의 턴어...,경제
3,본설 '배달 패키지' 본설 '배달 패키지'\n\n[파이낸셜뉴스] 외식 배달 전성시대...,경제
4,신종 코로나바이러스 감염증 공포가 확산하는 가운데 29일 인천국제공항 제1터미널 입...,경제
...,...,...
1595,중소벤처기업부는 19일 서울 여의도 중소기업중앙회 구내 나들가게에서 '스마트슈퍼 2...,IT/과학
1596,[서울=뉴시스] 이재은 기자 = 국내 대기업과 중견기업 상당수가 정부의 디지털 뉴딜...,IT/과학
1597,"(지디넷코리아=이도원 기자)넷마블(대표 권영식, 이승원)은 모바일 다중접속역할수행게...",IT/과학
1598,SK텔레콤에서 AI 개발을 담당하고 있는 현직 전문가들이 19일 온라인으로 진행된 ...,IT/과학


# 데이터 전처리

In [6]:
df1 = df.copy()

* news column의 **문자열**을 한글이 아닌것(^)모두 빈칸으로

In [7]:

df1['news'] = df['news'].str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ]','')

In [8]:
df1

,news,code
0,파이낸셜뉴스고가주택에대한대출규제를담은대책이후서울아파트시장이양극화되는분위기다고가아파트...,경제
1,플룸테크코리아제공플룸테크코리아제공파이낸셜뉴스전자담배시장이앞으로더확대될것으로전망된다냄...,경제
2,파이낸셜뉴스이아이디가차전지자회사의실적호전에힘입어지난해큰폭의턴어라운드에성공했다이아이디...,경제
3,본설배달패키지본설배달패키지파이낸셜뉴스외식배달전성시대를맞았다공정거래위원회에따르면배달앱...,경제
4,신종코로나바이러스감염증공포가확산하는가운데일인천국제공항제터미널입국장에서중국텐진발입국한...,경제
...,...,...
1595,중소벤처기업부는일서울여의도중소기업중앙회구내나들가게에서스마트슈퍼호점개점행사를열었다지난...,IT/과학
1596,서울뉴시스이재은기자국내대기업과중견기업상당수가정부의디지털뉴딜정책을긍정적으로생각한다는조...,IT/과학
1597,지디넷코리아이도원기자넷마블대표권영식이승원은모바일다중접속역할수행게임블레이드앤소울레볼루...,IT/과학
1598,텔레콤에서개발을담당하고있는현직전문가들이일온라인으로진행된커리큘럼라이브에참여하고있다텔레...,IT/과학


* 데이터에 null존재 확인

In [17]:
df1.isnull().sum()

news    0
code    0
dtype: int64

* 중복된 샘플 제거

In [18]:
df1.drop_duplicates(subset=['news'], inplace=True, ignore_index=True)

* 중복 제거결과 18820개의 뉴스가 15455개로 정리됨

In [20]:
print(df1['news'][0])

파이낸셜뉴스고가주택에대한대출규제를담은대책이후서울아파트시장이양극화되는분위기다고가아파트가밀집한강남권은매수심리가위축되면서재건축중심의하락세를보였다반면비강남권의경우중저가아파트가많은노원관악도봉구등에수요가유입되면서집값오름세를이어갔다수도권도교통호재로강남권접근성이개선되는수원용인등경기남부권일부지역을중심으로집값이큰폭으로올랐다일부동산에따르면이번주서울아파트매매가격은를기록했다재건축은강남구대치동은마와송파구잠실동주공단지등주요단지가하향조정되면서떨어졌다일반아파트는올라상승폭이둔화됐다신도시는올랐다경기인천은중저가아파트로수요가유입되면서상승했다전세시장은신종코로나코로나의영향으로국지적으로문의가줄었지만서울은매물품귀가지속되며상승했다신도시는올라전주와비슷한흐름을이어갔다경기인천은상승해지난주보다오름폭이커졌다서울은비강남권의상승세가이어지면서지역별로노원관악성북도봉광진강서구로순으로올랐다노원은광운대역세권개발동북선경전철호재가있는월계동미륭미성삼호차를비롯해상계동주공단지불암현대등이만원만원상승했다반면고가아파트밀집지역인송파동작강남은떨어졌다송파는잠실동주공단지와레이크팰리스신천동잠실파크리오가만원만원하락했다동작은년입주한흑석동아크로리버하임이만원하향조정됐다강남은대치동은마한보미도맨션차등재건축단지가만원만원내렸다신도시는동탄광교분당평촌산본순으로상승했다동탄은청계동시범예미지와반송동메타폴리스능동동탄숲속마을자연앤경남아너스빌이중대형면적위주로만원만원올랐다광교는매물이소진된후원천동광교호반베르디움하동광교레이크파크한양수자인이만원만원상승했다경기인천은신분당선남부연장광교중앙호매실예비타당성통과로교통개선에대한기대감이커진지역들이오름세를견인했다지역별로는수원용인의왕광명안양인천순으로올랐다수원은교통호재에따른기대감으로거래가이어지면서호매실동호매실와화서동한진현대등이만원만원상승했다용인은마북동삼거마을삼성래미안차성복동성동마을빌리지차중동신동백서해그랑블차등이만원만원상승했다최근정부는대책이후집값과열양상을보이는수도권일부지역의주택시장동향을점검하고추가대책을검토했다이에신분당선등교통호재와원도심재개발사업으로집값이크게오른수원을비롯해용인성남등일부지역이규제대상에포함될것으로보인다주택시장에대한모니터링강화와고가아파트거래자에대

In [21]:
print(df1['news'][1])

플룸테크코리아제공플룸테크코리아제공파이낸셜뉴스전자담배시장이앞으로더확대될것으로전망된다냄새가적다는것이가장큰장점인전자담배시장은년뒤국내에서조원규모로성장할전망이다최근기획재정부발표에따르면지난해전체담배판매량에서궐련형전자담배가차지하는비중이처음으로넘었다지난해월시장조사업체유로모니터인터내셔널의발표에서도년한국전체담배시장규모억만달러약조억원가운데궐련형전자담배시장이억만달러약조원으로집계됐다이는년억달러약억원보다배이상성장한것이다또년까지국내전체담배시장은억만달러약조억원으로소폭증가하지만전자담배시장은연평균씩성장해억만달러약조억원까지확대될것으로내다봤다유로모니터는새로운기기가나오면크게주목하고기기를잘다루며최신기술에열광하는한국소비자특성이한국전자담배시장의성장을이끌었다고분석했다코리아의플룸테크는액상카트리지와타바코캡슐을결합해사용하는하이브리드형전자담배다저온가열기술을적용해냄새발생을대폭줄였다캡슐만교체하면돼사용후에도담뱃재나찌꺼기가생기지않는다궐련형전자담배로는고형물을고열로가열해니코틴증기를흡입하는방식으로의릴한국필립모리스의아이코스브리티시아메리칸타바코의글로등이있다액상형전자담배도있지만최근유해성논란으로인해정부가사용중단을권고해시장축소위기에놓였다저작권자파이낸셜뉴스무단전재재배포금지


# 데이터 탐색

In [22]:
df1.code.value_counts(ascending=True)

IT/과학    3475
사회       3931
경제       3950
생활/문화    4099
Name: code, dtype: int64

In [23]:
# 데이터 실제 갯수 확인
# size() - 각 그룹의 사이즈를 취득할 수 있습니다.
print(df1.groupby('code').size().reset_index(name='count'))

    code  count
0  IT/과학   3475
1     경제   3950
2     사회   3931
3  생활/문화   4099


* 토큰화

In [24]:
from konlpy.tag import Okt, Hannanum, Kkma, Komoran

In [25]:
tokenizer1 = Komoran()

In [26]:
tokenizer1.morphs

<bound method Komoran.morphs of <konlpy.tag._komoran.Komoran object at 0x7f5fb9699b10>>

* 불용어 제거

In [36]:
# def preprocess(data):
#     text_data=[]
#     for i in data:
#         word_data = []
#         word_data = tokenizer1.morphs(i)
#         print(word_data)
#         text_data.append(word_data)
        
#     return text_data
   
    
    

In [27]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [28]:
def preprocessing(data):
    text_data=[]
    
    for sentence in data:
        temp_data=[]
        # 토큰화
        temp_data = tokenizer1.morphs(sentence)
        
        # 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)
        
    text_data = list(map(' '.join, text_data))  #.join -- list to str
    
    return text_data
    

In [30]:
preprocessing(df1.news.loc[:5])

['파이낸셜뉴스고가주택에대한대출규제를담은대책이후서울아파트시장이양극화되는분위기다고가아파트가밀집한강남권은매수심리가위축되면서재건축중심의하락세를보였다반면비강남권의경우중저가아파트가많은노원관악도봉구등에수요가유입되면서집값오름세를이어갔다수도권도교통호재로강남권접근성이개선되는수원용인등경기남부권일부지역을중심으로집값이큰폭으로올랐다일부동산에따르면이번주서울아파트매매가격은를기록했다재건축은강남구대치동은마와송파구잠실동주공단지등주요단지가하향조정되면서떨어졌다일반아파트는올라상승폭이둔화됐다신도시는올랐다경기인천은중저가아파트로수요가유입되면서상승했다전세시장은신종코로나코로나의영향으로국지적으로문의가줄었지만서울은매물품귀가지속되며상승했다신도시는올라전주와비슷한흐름을이어갔다경기인천은상승해지난주보다오름폭이커졌다서울은비강남권의상승세가이어지면서지역별로노원관악성북도봉광진강서구로순으로올랐다노원은광운대역세권개발동북선경전철호재가있는월계동미륭미성삼호차를비롯해상계동주공단지불암현대등이만원만원상승했다반면고가아파트밀집지역인송파동작강남은떨어졌다송파는잠실동주공단지와레이크팰리스신천동잠실파크리오가만원만원하락했다동작은년입주한흑석동아크로리버하임이만원하향조정됐다강남은대치동은마한보미도맨션차등재건축단지가만원만원내렸다신도시는동탄광교분당평촌산본순으로상승했다동탄은청계동시범예미지와반송동메타폴리스능동동탄숲속마을자연앤경남아너스빌이중대형면적위주로만원만원올랐다광교는매물이소진된후원천동광교호반베르디움하동광교레이크파크한양수자인이만원만원상승했다경기인천은신분당선남부연장광교중앙호매실예비타당성통과로교통개선에대한기대감이커진지역들이오름세를견인했다지역별로는수원용인의왕광명안양인천순으로올랐다수원은교통호재에따른기대감으로거래가이어지면서호매실동호매실와화서동한진현대등이만원만원상승했다용인은마북동삼거마을삼성래미안차성복동성동마을빌리지차중동신동백서해그랑블차등이만원만원상승했다최근정부는대책이후집값과열양상을보이는수도권일부지역의주택시장동향을점검하고추가대책을검토했다이에신분당선등교통호재와원도심재개발사업으로집값이크게오른수원을비롯해용인성남등일부지역이규제대상에포함될것으로보인다주택시장에대한모니터링강화와고가아파트거래자